In [1]:
#Daniel Castaneda Mogollon, PhD
#January 20, 2025
#This script was generated to compare the w5w6 and w9w10 data, take the ASVs that were not identified at the
# species level (Vsearch 99.3%) or at the genus level (DADA2 97%) in the local database (my full-length 16S Sanger
#+ 16S WGS) and retrieved from the GTDB from FemMicro16S to see if I have external contaminants.

In [2]:
import pandas as pd

In [40]:
#This section reads the local file of unidentified asvs and the femmicro file
df_w9 = pd.read_csv("unidentified_asvs_by_dada2_and_vsearch_w9w10.csv")
df_w9_femmicro = pd.read_csv("../FemMicro16S/femmicro_output_w9w10/vsearch_dada2_gtdb_ure_merged.tsv",sep='\t') #Had to manually add the ASV_ID Column
df_w9_list = df_w9['ASV_ID'].tolist()
df_w9_femmicro_list = df_w9_femmicro['asv_num'].tolist()

,Unnamed: 0,ASV_ID
0,213,ASV_214
1,383,ASV_384
2,433,ASV_434
3,473,ASV_474
4,512,ASV_513
...,...,...
80,1997,ASV_1998
81,1998,ASV_1999
82,1993,ASV_1994
83,2022,ASV_2023


In [62]:
#This section creates a list of indices of the unidentified asvs in my local db output file (vsearch + dada2)
index_list=[]
for items in df_w9_list:
    if items in df_w9_femmicro_list:
        items_index = df_w9_femmicro_list.index(items)
        index_list.append(items_index)

In [70]:
#This section takes five columns of the femmicro output file for taxonomy and subsets the asvs not identified by
#my local database for me to analyze, then it writes it in an output file
filtered_df = df_w9_femmicro.iloc[index_list]
filtered_df = filtered_df[['asv_num','genus_final','database_final','package','database_vsearch']]
femmicro_genus = filtered_df['genus_final'].tolist()
femmicro_genus = list(set(femmicro_genus))
filtered_df.to_csv('femmicro_identified_asvs_not_found_in_local.csv')


In [73]:
#This section takes the filtered master file for downstream selection, and then it filters by genus from GTDBtk pplacer
#Then it saves it into a list with unique genera (no repetitions)
master_unique_genus_list = []
master_file = pd.read_csv("../WGS_all_runs/FINAL_report_WGS021224.csv")
master_file_filtered = master_file.loc[master_file['Selected for Downstream']=='Yes',['Classification']]
master_taxa_list = master_file_filtered['Classification'].tolist()
for items in master_taxa_list:
    items = items.split("g__")[1]
    items = items.split(";")[0]
    master_unique_genus_list.append(items)
master_unique_genus_list = list(set(unique_genus_list))

In [74]:
#This section prints the genus that was identified in FemMicro16S of the ASVs not found locally (dada2 + vsearch)
#and that were not present in the Master file with the latest nomenclature system (r220):
for item in femmicro_genus:
    if item not in master_unique_genus_list:
        print(item)

Clostridium_P
Prevotella
Hungatella_A
nan


In [95]:
#This section takes the Prevotella assignment from the original FemMicro16S output, and filters by Prevotella. 
#Then, I look at the ASVs assigned to Prevotella and retrieve them from the vsearch local output, where 85 were
#assigned to Phocaeicola vulgatus
index_prevotella_list=[]
df_w9_femmicro_filtered = df_w9_femmicro[['asv_num','genus_final']]
df_w9_femmicro_filtered_prevotella = df_w9_femmicro.loc[df_w9_femmicro['genus_final']=='Prevotella',['asv_num','genus_final']]
df_w9_femmicro_filtered_prevotella_list = df_w9_femmicro_filtered_prevotella['asv_num'].tolist()
df_local_vsearch = pd.read_csv("output/vsearch_taxonomy_w9w10_output_993.tsv",sep='\t')
#print(df_local_vsearch)
df_local_vsearch_asv_list = df_local_vsearch['asv_id'].tolist()
for items in df_w9_femmicro_filtered_prevotella_list:
    if items in df_local_vsearch_asv_list:
        index_prevotella_list.append(df_local_vsearch_asv_list.index(items))
vsearch_prevotella = df_local_vsearch.iloc[index_prevotella_list]
vsearch_prevotella.to_csv("output/prevotella_genus_by_femmicro_w9.csv")

In [102]:
index_prevotella_list_dada2=[]
df_local_dada2= pd.read_csv('output/dada2_assignTaxonomy_w9w10_rdp.csv')
df_local_dada2_asv_list = df_local_dada2['ASV_ID'].tolist()
for items in df_w9_femmicro_filtered_prevotella_list:
    if items in df_local_dada2_asv_list:
        index_prevotella_list_dada2.append(df_local_dada2_asv_list.index(items))
dada2_prevotella = df_local_dada2.iloc[index_prevotella_list_dada2]
dada2_prevotella.to_csv("output/prevotella_genus_by_dada2_local_w9.csv")